In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
import scipy.stats

In [ ]:
rng = RandomState(20190401)
orig = rng.rand(15,2)
a =  orig * np.array([0.5,2])[None]

b = orig  * np.array([2,0.5])[None] + np.array([2,3])[None]

plt.figure(figsize=(5,5))
plt.plot(a[:,0], a[:,1], ls='', marker='o')
plt.plot(b[:,0], b[:,1], ls='', marker='o', color=seaborn.color_palette()[4],
        mfc='none')

for one_a, one_b in zip(a,b):
    plt.annotate("",
                 xy=one_b, xytext=one_a,
               arrowprops=dict(arrowstyle="->",),
                )
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend(("Inputs", "Targets"))

In [ ]:
from torch import nn
from reversible.revnet import ReversibleBlockOld
def dense_rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2,bias=True)),
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2, bias=True))
    )

In [ ]:
from reversible.revnet import ReversibleBlockOld

from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_rev_block(2,200),
    dense_rev_block(2,200),
)
inputs = np_to_var(a, dtype=np.float32)
targets = np_to_var(b, dtype=np.float32)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 20001
for i_epoch in range(n_epochs):
    outs = feature_model(inputs)
    loss = th.mean((outs -targets) ** 2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs //20) == 0:
        print("Epoch {:d}".format(i_epoch))
        print("Loss {:.3f}".format(loss.item()))
        outs = feature_model(inputs)

        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(outs[:,0]), var_to_np(outs[:,1]), ls='', marker='o')
        plt.plot(var_to_np(targets[:,0]), var_to_np(targets[:,1]), ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)

In [ ]:
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step " +  str(i) for i in range(5)] + ["Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

plt.xlim(0,4)
plt.ylim(-4,5)

In [ ]:
fig = plt.figure(figsize=(5,5))
for i_inv, this_inv in enumerate(all_inverted):
    
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    pass

In [ ]:
from affine import AffineBlock

In [ ]:
from torch import nn
from reversible.revnet import ReversibleBlockOld
def dense_aff_block(n_c, n_i_c):
     return AffineBlock(
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2,bias=True)),
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2, bias=True)),
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2,bias=True)),
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2,bias=True))
     )


In [ ]:
from reversible.revnet import ReversibleBlockOld

from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_aff_block(2,200),
    dense_aff_block(2,200),
)
inputs = np_to_var(a, dtype=np.float32)
targets = np_to_var(b, dtype=np.float32)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step " +  str(i) for i in range(5)] + ["Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

plt.xlim(0,4)
plt.ylim(-4,5)

In [ ]:
n_epochs = 20001
for i_epoch in range(n_epochs):
    outs = feature_model(inputs)
    loss = th.mean((outs -targets) ** 2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs //20) == 0:
        print("Epoch {:d}".format(i_epoch))
        print("Loss {:.3f}".format(loss.item()))
        outs = feature_model(inputs)

        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(outs[:,0]), var_to_np(outs[:,1]), ls='', marker='o')
        plt.plot(var_to_np(targets[:,0]), var_to_np(targets[:,1]), ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)

In [ ]:
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step " +  str(i) for i in range(5)] + ["Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

plt.xlim(0,4)
plt.ylim(-4,5)

In [ ]:
from torch import nn
from reversible.revnet import ReversibleBlockOld
def dense_mult_block(n_c, n_i_c):
     return AffineBlock(
       lambda x: x* 0,
       lambda x: x* 0,
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2,bias=True)),
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, bias=True),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2, bias=True)),
     )


### Pure Multiplicative as toy test

In [ ]:
from reversible.revnet import ReversibleBlockOld

from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_mult_block(2,200),
    dense_mult_block(2,200),
)
inputs = np_to_var(a, dtype=np.float32)
targets = np_to_var(b, dtype=np.float32)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step " +  str(i) for i in range(5)] + ["Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

plt.xlim(0,4)
plt.ylim(-4,5)

In [ ]:
n_epochs = 20001
for i_epoch in range(n_epochs):
    outs = feature_model(inputs)
    loss = th.mean((outs -targets) ** 2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs //20) == 0:
        print("Epoch {:d}".format(i_epoch))
        print("Loss {:.3f}".format(loss.item()))
        outs = feature_model(inputs)

        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(outs[:,0]), var_to_np(outs[:,1]), ls='', marker='o')
        plt.plot(var_to_np(targets[:,0]), var_to_np(targets[:,1]), ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step " +  str(i) for i in range(5)] + ["Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

plt.xlim(0,4)
plt.ylim(-4,5)

## Add then mult

In [ ]:
from reversible.revnet import ReversibleBlockOld

from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_rev_block(2,200),
    dense_mult_block(2,200),
)
inputs = np_to_var(a, dtype=np.float32)
targets = np_to_var(b, dtype=np.float32)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 20001
for i_epoch in range(n_epochs):
    outs = feature_model(inputs)
    loss = th.mean((outs -targets) ** 2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs //20) == 0:
        print("Epoch {:d}".format(i_epoch))
        print("Loss {:.3f}".format(loss.item()))
        outs = feature_model(inputs)

        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(outs[:,0]), var_to_np(outs[:,1]), ls='', marker='o')
        plt.plot(var_to_np(targets[:,0]), var_to_np(targets[:,1]), ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step " +  str(i) for i in range(5)] + ["Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

#plt.xlim(0,4)
#plt.ylim(-4,5)

## Mult then add

In [ ]:
from reversible.revnet import ReversibleBlockOld

from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_mult_block(2,200),
    dense_rev_block(2,200),
)
inputs = np_to_var(a, dtype=np.float32)
targets = np_to_var(b, dtype=np.float32)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 20001
for i_epoch in range(n_epochs):
    outs = feature_model(inputs)
    loss = th.mean((outs -targets) ** 2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs //20) == 0:
        print("Epoch {:d}".format(i_epoch))
        print("Loss {:.3f}".format(loss.item()))
        outs = feature_model(inputs)

        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(outs[:,0]), var_to_np(outs[:,1]), ls='', marker='o')
        plt.plot(var_to_np(targets[:,0]), var_to_np(targets[:,1]), ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step " +  str(i) for i in range(5)] + ["Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

#plt.xlim(0,4)
#plt.ylim(-4,5)

### only 1x1

In [ ]:
from reversible.revnet import ReversibleBlockOld

from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet
from conv1x1 import Conv1x1, Dense1x1

set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    Dense1x1(2),
)
inputs = np_to_var(a, dtype=np.float32)
targets = np_to_var(b, dtype=np.float32)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step " +  str(i) for i in range(5)] + ["Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

#plt.xlim(0,4)
#plt.ylim(-4,5)

In [ ]:
n_epochs = 20001
for i_epoch in range(n_epochs):
    outs = feature_model(inputs)
    loss = th.mean((outs -targets) ** 2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs //20) == 0:
        print("Epoch {:d}".format(i_epoch))
        print("Loss {:.3f}".format(loss.item()))
        outs = feature_model(inputs)

        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(outs[:,0]), var_to_np(outs[:,1]), ls='', marker='o')
        plt.plot(var_to_np(targets[:,0]), var_to_np(targets[:,1]), ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Input", "1x1Conv", "Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

#plt.xlim(0,4)
#plt.ylim(-4,5)

## with add block

In [ ]:
from reversible.revnet import ReversibleBlockOld

from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet
from conv1x1 import Conv1x1, Dense1x1

set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    Dense1x1(2),
    dense_rev_block(2,200),
    dense_rev_block(2,200),
)
inputs = np_to_var(a, dtype=np.float32)
targets = np_to_var(b, dtype=np.float32)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 10001
for i_epoch in range(n_epochs):
    outs = feature_model(inputs)
    loss = th.mean((outs -targets) ** 2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs //20) == 0:
        print("Epoch {:d}".format(i_epoch))
        print("Loss {:.3f}".format(loss.item()))
        outs = feature_model(inputs)

        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(outs[:,0]), var_to_np(outs[:,1]), ls='', marker='o')
        plt.plot(var_to_np(targets[:,0]), var_to_np(targets[:,1]), ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')
for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step 0", "Step 1 (1x1)"] +  ["Step " + str(i) + " (Additive)" for i in range(2,6)] + [
    "Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

#plt.xlim(0,4)
#plt.ylim(-4,5)

In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True)[::-1]
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')

plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')
plt.legend(["Step " +  str(i) for i in range(6)] + ["Targets"], bbox_to_anchor=(1,1,0,0))
plt.xlabel("x1")
plt.ylabel("x2")

#plt.xlim(0,4)
#plt.ylim(-4,5)

## With invertible residual Lipschitz Block

In [ ]:
feature_model = nn.Sequential(
    ResidualBlock(
        nn.Sequential(
        spectral_norm(nn.Linear(2,200), to_norm=0.92, n_power_iterations=3),
             nn.ReLU(),
             spectral_norm(nn.Linear(200,2), to_norm=0.92, n_power_iterations=3),
    )),
    ResidualBlock(
            nn.Sequential(
            spectral_norm(nn.Linear(2,200), to_norm=0.92, n_power_iterations=3),
                 nn.ReLU(),
                 spectral_norm(nn.Linear(200,2), to_norm=0.92, n_power_iterations=3),
        )),
    ResidualBlock(
            nn.Sequential(
            spectral_norm(nn.Linear(2,200), to_norm=0.92, n_power_iterations=3),
                 nn.ReLU(),
                 spectral_norm(nn.Linear(200,2), to_norm=0.92, n_power_iterations=3),
        )),)

inputs = np_to_var(a, dtype=np.float32)
targets = np_to_var(b, dtype=np.float32)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 10001
for i_epoch in range(n_epochs):
    outs = feature_model(inputs)
    loss = th.mean((outs -targets) ** 2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs //20) == 0:
        print("Epoch {:d}".format(i_epoch))
        print("Loss {:.3f}".format(loss.item()))
        outs = feature_model(inputs)

        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(outs[:,0]), var_to_np(outs[:,1]), ls='', marker='o')
        plt.plot(var_to_np(targets[:,0]), var_to_np(targets[:,1]), ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
th.mean(th.abs(all_inverted[0] - inputs))

In [ ]:
len(all_inverted)

In [ ]:
outs = feature_model(inputs)
all_inverted = invert(feature_model, outs, return_all=True, residual_iterations=50)[::-1]
print("MAE: {:E}".format(th.mean(th.abs(all_inverted[0] - inputs))))
len(all_inverted)
fig = plt.figure(figsize=(4.5,4.5))
for i_inv, this_inv in enumerate(all_inverted):
    pts = var_to_np(this_inv)
    plt.plot(pts[:,0], pts[:,1], ls='', marker='o')

for prev_inv, next_inv in zip(all_inverted[:-1], all_inverted[1:]):
    for one_prev, one_next in zip(var_to_np(prev_inv), var_to_np(next_inv)):
        plt.annotate("",
                     xy=one_next, xytext=one_prev,
                   arrowprops=dict(arrowstyle="->",),
                    )
plt.plot(var_to_np(targets)[:,0], var_to_np(targets)[:,1], ls='', 
        color=seaborn.color_palette()[4], marker='o', markerfacecolor='none')

plt.legend(["Step " +  str(i) for i in range(4)] + ["Targets"], bbox_to_anchor=(1,1,0,0))

plt.xlabel("x1")
plt.ylabel("x2")

#plt.xlim(0,4)
#plt.ylim(-4,5)